# CS369 Assignment 2

## _Ben Halstead, ID: 8345337_

## Problem 1


### A)

To simulate a sequences, i used two classes, one for each site and one for a sequence. I also made a class to generate a sequence of a given length with a uniform distribution.

In [1]:
import math, random
#easy way to map number(as index) to base name
BASES = ['A','C','G','T']
class sequence:
    #Class for a sequence of sites, has a method to check for mutations at a single given site
    # the SiteIndex is used for problem 2, for insertions and deletions (printing the gaps mostly)
	def __init__(self, startBases):
		self.bases = []
		self.length = 0
		for i in startBases:
			#can make with a str for type or a tuple of type and siteIndex
			if(type(i) == type(str())):
				nB = base(i, self.length)
			elif(type(i) == type(tuple())):
				nB = base(i[0],i[1])
			self.bases.append(nB)
			self.length += 1
	def add(self, i, nB):
		self.bases.insert(i, base(nB,i))
		self.length += 1
		for j in range(i+1,self.length):
			self.bases[j].siteIndex = j
	def insert(self, i, seq):
		#insert a sequence after position i
		for j in range(0,seq.length):
			seq.bases[j].siteIndex = -1
		self.bases[i:i] = seq.bases
		self.length += seq.length
	def deleteSeq(self, i):
		#delete site i and the next 2 after
		numDeletions = min(3, self.length-i)
		for index in range(numDeletions):
			self.bases.pop(i)
			self.length -= 1
	def stringRepr(self):
		stringR = ""
		for b in self.bases:
			stringR+=b.__str__()
		return stringR
	def checkMutation(self, i, u):
        #checks to see if a site should mutate.
		rand = random.random()
		b = self.bases[i]
		if rand <= u:
			r = self.bases[i].mutate()
			return r
		else:
			return False

	def getType(self, i):
		return self.bases[i].type
	def copy(self):
		newSeq = []
		for i in self.bases:
			newSeq.append((i.type,i.siteIndex))
		return sequence(newSeq)
	def __str__(self):
		return str(self.bases)
	def __repr__(self):
		return self.__repr__()
	def __len__(self):
		return self.length

class base:
    #Class for a single site, method to mutate into a different base
	def __init__(self, initial, sI):
		if(initial in BASES):
			self.siteIndex = sI
			self.type = initial
			self.tIndex = BASES.index(initial)

	def mutate(self):
        #picks which new base to mutate into
		newIndex = math.floor(random.random()*4)
        #only mutates if it would change (3/4 probability)
		if(newIndex != self.tIndex):
			self.tIndex = newIndex
			self.type = BASES[newIndex]
			return True
		else:
			return False

	def __str__(self):
		#return self.type+":"+str(self.siteIndex)
		return str(self.type)
	def __repr__(self):
		return self.__str__()
    
def makeSeq(length):
    #Constructs a uniform random sequence of bases
	se = []
	for i in range(0,length):
		j = math.floor(random.random()*4)
		se.append(BASES[j])
	return sequence(se)

function q1 then uses these to simulate the evolution of two sequences from a common ancestor.
A random sequence with uniform distribution is created with a given length, then two copies are made which will both be mutated separatly.
For each timestep until the given ending time, each site in a child sequence has the checkMutation method called on it. If a mutation is found to have occured (chance $u$), the site has the mutate method called. This method Mutates the site with a $\frac{3}{4}$ chance of changing, and a $\frac{1}{4}$ chance of not changing and just returning. This gives an overall chance for each site in a sequence, per timestep, to mutate of $ \frac{3}{4}u$
After all timesteps are complete, the sequences are then compared to the ancestor and to each other, and the results printed.


In [21]:
def q1(sLength, t, u):
    #make initial ancestor sequence
    seq = makeSeq(sLength)
    #start children sequences
    s1 = seq.copy()
    s2 = seq.copy()
    s1DiffAnc = 0
    s1DiffSib = 0
    s2DiffAnc = 0
    s2DiffSib = 0
    #for each timestep
    for j in range(0,t):
        #for each site in sequences
        for i in range(0,sLength):
            #check if mutation occurs, will call mutate on bases if so
            s1M = s1.checkMutation(i,u)
            s2M = s2.checkMutation(i,u)
    return seq,s1,s2
def compareSeq(s1,s2):
    diffs = 0
    #check for differences site by site
    for k in range(0,len(s1)):
        #check s1 against s2
        if s1.getType(k) != s2.getType(k):
            diffs += 1
    return diffs

In [27]:
q1Anc, q1Child1, q1Child2 = q1(50, 10, 0.01)
print("Ancestor:\t" + str(q1Anc.stringRepr()))
print("Sequence 1:\t" + str(q1Child1.stringRepr()))
print("Sequence 2:\t" + str(q1Child2.stringRepr()))
print("S1 to Ancestor differences: " + str(compareSeq(q1Anc,q1Child1)))
print("S2 to Ancestor differences: " + str(compareSeq(q1Anc,q1Child2)))
print("S1 to S2 differences: " + str(compareSeq(q1Child1,q1Child2)))

Ancestor:	CTTTATCAGTCAATTACGATGGTCCTACGTAGACCCGGGTACCCATCAGC
Sequence 1:	CTTTAACAGGCAATCACGATGGTCCTACGTAGACCCGGGTACCCATCAGC
Sequence 2:	CTTTATCATTCAATTACGAGGGTCTTACGTAGACCCGGGTACACATCTAC
S1 to Ancestor differences: 3
S2 to Ancestor differences: 6
S1 to S2 differences: 9


### B)

The Poisson distribution describes the probability of a given number of events occuring in a fixed interval of time. Since the number of mutations are independantly occuring events over a period of time, the number of them should be a poisson distribution.
On each timestep, each site has a $u$ chance to mutate. Since there is a $\frac{1}{4}$ chance to mutate to the same base, so overall a $\frac{3}{4}u$ chance for each base to mutate on each timestep. Since there are $L$ sites and 2 sequences, this is a total of $2L\frac{3}{4}u$ number of mutations for each timestep. Since there are $t$ timesteps, the expected value of mutations is $2tL\frac{3}{4}u$ overall, so this will be the Poisson distribution parameter.

In [4]:
def q2():
    mean = 0
    difs = []
    #1000 pairs simulated
    for i in range(0,1000):
        #just calls the same function as in q1 to create pairs. d is then the difference between siblings.
        q1Return = q1(1000, 25, 0.01)
        #set d to be the returned sibling difference from q1
        d = q1Return[5]
        #adds difference to list
        difs.append(d)
        mean += d

    sumSquaresM = 0
    sumSquares = 0
    mean = mean/1000
    
    #working out variance
    for k in range(0, 1000):
        sumSquares += pow(difs[k] - mean, 2)

    sumSquares = sumSquares/1000


    print("Mean: " + str(mean))
    print("Variance: " + str(sumSquares))


In [5]:
q2()

Mean: 295.637
Variance: 198.479231


In a poisson distribution, the mean and variance should be equal. A Poisson distribution with rate $2tL\frac{3}{4}u$ with $t=25, u=0.01, l=1000$ would be have $mean = variance = 375$ which is not what we get here. This means the number of differences between pairs is not poisson distributed.

### C)

The function i used to simulate the sequences for c is:

In [59]:
def q3():
	pABMap = {}
	#Evolves the sibling sequences
	q1Return = q1(10000,10,0.03)
	s1 = q1Return[1]
	s2 = q1Return[2]
	#for every pair of sites, add the combination to the count map
	for i in range(0,s1.length):
		a = s1.getType(i)
		b = s2.getType(i)
		if((a,b) not in pABMap):
			pABMap[(a,b)] = 0
		pABMap[(a,b)] += 1
	#print(pABMap)
	F = []
	#Construct probability matrix
	for i,r in enumerate(BASES):
		F.append([])
		for c in BASES:
			F[i].append(pABMap[(r,c)])
	#print(F)
	#Change F from counts to row probabilities
	for r in range(0,len(F)):
		rowTotal = sum(F[r])
		for c in range(0, len(F[0])):
			F[r][c] = F[r][c]/rowTotal
	#print(F)
	print("P_ab when a = b theoretical: " +str(round(((1/4)+(3/4)*math.exp(-2*10*0.03))*100)/100))
	print("P_ab when a != b theoretical: " +str(round(((1/4)-(1/4)*math.exp(-2*10*0.03))*100)/100))
	print("Estimates: ")
	topRow = ""
	for k in BASES:
		topRow += ("\t"+k)
	print(topRow)
	for r in range(0,len(F)):
		rowStr = BASES[r]
		for c in range(0, len(F[0])):
			rowStr += "\t"+str(round(F[r][c]*100)/100)
		print(rowStr)

In [60]:
q3()

P_ab when a = b theoretical: 0.66
P_ab when a != b theoretical: 0.11
Estimates: 
	A	C	G	T
A	0.67	0.12	0.11	0.11
C	0.11	0.66	0.12	0.11
G	0.11	0.12	0.66	0.11
T	0.12	0.11	0.12	0.66


My estimates are ~ the theoretical values, so are most likly correct.

## Problem 2

For the insertions and deletions, I made two functions, which take in a sequence, the ancestor length, time and u. Using a function based on knuth's algorithm to generate a poisson random variable, the functions get a random number of insertions or deletions, then work out the indexs where these should occur (chosen uniform randomly from the original sequence). these indexes are then put into decending order, so when insertions and deletions occur it doesnt mess up the ones after, since lower indexes will be uneffected. 

In [8]:
def poissonRV(para):
	#using knuth's algorithm for generating a poisson distributed number.
	L = math.exp(para*-1)
	k = 0
	p = 1
	while p> L:
		u = random.random()
		k += 1
		p = p*u
	return k-1

In [78]:
def insertions(seq, sLength, t, u):
	#draw a poisson random variate with rate Ltu/10 to give us the number of insertions
	numInsertions = poissonRV(sLength*t*u/10)
	newSeq = seq.copy()
	for i in range(0,numInsertions):
		randI = math.floor(random.random()*seq.length)
		#make a uniform random sequence of length 3
		inSeq = makeSeq(3)
		#insert in position after index
		newSeq.insert(randI+1, inSeq)
	return (newSeq,numInsertions)
	#insertionIndexs = []
	##find the indexes of the original we will be adding to. These are uniformly randomly chosen
	#for j in range(0,numInsertions):
	#	randI = math.floor(random.random()*seq.length)
	#	insertionIndexs.append(randI)
	##put indexes in decending order so when inserting it doesnt mess up index number
	#insertionIndexs.sort()
	#insertionIndexs.reverse()
	#newSeq = seq.copy()
	#for index in insertionIndexs:
	#	#make a uniform random sequence of length 3
	#	inSeq = makeSeq(3)
	#	#insert in position after index
	#	newSeq.insert(index+1, inSeq)

	#return (newSeq,numInsertions)

def deletions(seq, sLength, t, u):
	#draw a poisson random variate with rate Ltu/10 to give us the number of deletions
	numDeletions = poissonRV(sLength*t*u/10)
	newSeq = seq.copy()
	for i in range(0,numDeletions):
		randI = math.floor(random.random()*seq.length)
		#make a uniform random sequence of length 3
		inSeq = makeSeq(3)
		#Delete from index
		newSeq.deleteSeq(randI)
	return (newSeq,numDeletions)

	#deletionIndexs = []
	##find the indexes of the original we will be deleting from. These are uniformly randomly chosen
	#for j in range(0,numDeletions):
	#	randI = math.floor(random.random()*seq.length)
	#	deletionIndexs.append(randI)
	##put indexes in decending order so when deleting it doesnt mess up index number
	#deletionIndexs.sort()
	#deletionIndexs.reverse()
	#newSeq = seq.copy()
	#for index in deletionIndexs:
	#	#make a uniform random sequence of length 3
	#	inSeq = makeSeq(3)
	#	#Delete from index
	#	newSeq.deleteSeq(index)

	#return (newSeq,numDeletions)

To print out the sequence showing where insertions and deletions occur uses the siteIndex value for each base. A pointer is initialised at the start of each sequence, and the siteIndex of the site pair is looked at. If the siteIndexes match, the sites must have also been paired in the original mutated sequences, so they are just printed as is and both pointers advance. 
If either has a siteIndex of -1, it is an inserted value, so that is printed and a '-' is printed for the other sequence. Only the pointer that printed a letter is advanced.
If either siteIndex is higher then the other, a deletion must have occured in that sequence, so a '-' is printed for that one and the letter for the other, only the other pointer is advanced.
Once the end of one sequence is reached, the rest of the other is printed, with a corrisponding gap printing for the finished sequence. However, the siteIndex of the unfinished sequence is still kept track of incase of deletions which will show up as a jump in siteIndex. in this case '-' are appended until the jump is finished.

In [79]:
def printWithChanges(s1,s2):
	a = 0
	b = 0
	aString = []
	bString = []
	while a < s1.length and b < s2.length:
		aSiteI = s1.bases[a].siteIndex
		bSiteI = s2.bases[b].siteIndex
		# If equal, the sites match so print out both
		if(aSiteI == bSiteI):
			aString.append(s1.getType(a))
			bString.append(s2.getType(b))
			a += 1
			b += 1
		# if one is an insertion, print it and a gap for other
		elif(aSiteI == -1):
			aString.append(s1.getType(a))
			bString.append('-')
			a += 1
		elif(bSiteI == -1):
			bString.append(s2.getType(b))
			aString.append('-')
			b += 1
		#if there is a deletion, one will have a higher site index then the other, so print the other and a gap for the sequence with deletions
		elif(aSiteI > bSiteI):
			#deletion in a
			bString.append(s2.getType(b))
			aString.append('-')
			b += 1
		elif(bSiteI>aSiteI):
			#deletion in b
			aString.append(s1.getType(a))
			bString.append('-')
			a += 1
	if( a == s1.length):
		# a sequence done, do add on rest of b with deletions
		prevSI = b
		for j in range(b+1, s2.length):
			nextIndex = s2.bases[j].siteIndex
			while nextIndex != prevSI+1:
				bString.append('-')
				prevSI+=1
			bString.append(s2.getType(nextIndex))
			aString.append('-')
			prevSI = j
	if( b == s2.length):
		# a sequence done,  add on rest of a with deletions
		prevSI = a
		for j in range(a+1, s1.length):
			nextIndex = s1.bases[j].siteIndex
			while nextIndex != prevSI+1:
				aString.append('-')
				prevSI+=1
			aString.append(s1.getType(nextIndex))
			bString.append('-')
			prevSI = j
	print(''.join(aString))
	print(''.join(bString))

Now, to simulate the new mutations the function is able to just create two mutated sibling sequences with the q1 code, then run the insertion then deletion functions on both. This creates two sibling sequences with all mutations, insertions and deletions.

In [94]:
def p2q1(sLength, t, u):
	#simulate mutations
    #loop in case there are no insertions or deletions, in which case we just try again
	while True:
		q1Return = q1(sLength, t, u)
		A = q1Return[1]
		B = q1Return[2]
		A,niA = insertions(A, sLength, t, u)
		A,ndA = deletions(A, sLength, t, u)
		B,niB = insertions(B, sLength, t, u)
		B,ndB = deletions(B, sLength, t, u)
		if(niA+ndA+niB+ndB != 0):
			break
	printWithChanges(A,B)
	return (A,B)

The insertion/deletion functions return the number of each that is performed, if the total sum of these is 0, the function is called again to ensure something interesting actually happens.
Otherwise, the print funtion is called to print the sequences.

In [93]:
seqA,seqB = p2q1(50, 20, 0.01)

ATTT---TCACCTG--CCCCATC---GAGGTCGACCCGCCGACGATTTCATAACCTTC
ATAAGACGCACCCGCT--GCATATGAGAGGCC---ACGCGGACGAG---ATCACCTTC


### B)

A poisson distribution models the occurances of rare events over time if the events are independant of eachother.With the model of insertions and deletions in 2A, the number of insertions and deletions is based off of the original L, so each I/D is Independant of eachother. For the model in 2B, after each insertion or deletion the length of the sequences are effected. Since the rate of overall insertions/deletions is dependant of the length of the sequence (each site has a chance to add an insertion or deletion, so more sites = more I/D), This means each event is effecting the events after it so the events are not independant of eachother, so the total number of insertons and deletions is _not_ poisson distributed. 

# Problem 3

### a)

My overlap alignment function is based on the given pseudocode. First the f matrix is initialised with len(s1)+1 rows and len(s2)+1 columns. The +1 is for the additional 0 row and column at the start. Each matrix value has a value, which is initialised with a 0 if in the first row or column (this is because this is an overlap alignment, so we dont penalize gaps at the start) or None otherwise. 
Each node in the matrix takes the form:
[value, previous node in path, print value for s1, print value for s2, [rowIndex, colIndex]]

Then the matrix is filled in according to the needleman-welch algorithm, taking the max match, gapS1 or gapS2 score and saving the previous node as the path. 

Then there is some extra code for looking at the left/bottom edge for the max value (This is to not penalize gaps at the end of the alignment) and traversing back through the path and printing each nodes print values (in reverse order).

Since the alignment is an overlap alignment, the aligned part might not start at the start or end at the end of both at the same time. Thus we need to print the extras aligned with gaps. This is what the code at the end is doing.


In [97]:
def overlapAlignment(s1,s2,score,gapPen):
	f = []
	for iR, j in enumerate([0]+s1):
		f.append([])
		for iC,k in enumerate([0]+s2):
			if(iR ==0 or iC == 0):
				f[iR].append([0,None,None,None,[iR,iC]])
			else:
				f[iR].append([None,None,None,None,[iR,iC]])

	for iC,k in enumerate(s2):
		for iR, j in enumerate(s1):
			iCol = iC+1
			iRow = iR+1
			matchLetterS1 = BASES.index(j)
			matchLetterS2 = BASES.index(k)
			matchScore = score[matchLetterS1][matchLetterS2]
            #gap penalty is subtracted, so given as a positive in input.
			Match = [f[iRow-1][iCol-1][0] + matchScore, f[iRow-1][iCol-1], j,k,[iRow,iCol]]
			GapS1 = [f[iRow-1][iCol][0] - gapPen, f[iRow-1][iCol], j,'-',[iRow,iCol]]
			GapS2 = [f[iRow][iCol-1][0] - gapPen, f[iRow][iCol-1], '-',k,[iRow,iCol]]

			maxTo = max([Match,GapS1,GapS2], key= lambda item:item[0])
			f[iRow][iCol] = maxTo

	endsList = []
	for r in range(0,len(f)):
		for c in range(0,len(f[0])):
			if(r == len(f)-1 or c == len(f[0])-1):
				endsList.append(f[r][c])
	maxEnd = max(endsList, key=lambda item:item[0])
	s1Path = []
	s2Path = []
	s1Path.append(maxEnd[2])
	s2Path.append(maxEnd[3])
	fromPointer = maxEnd[1]
	while fromPointer != None:
		s1Path.append(fromPointer[2])
		s2Path.append(fromPointer[3])
		if(fromPointer[1][2] == None):
			break
		fromPointer = fromPointer[1]
	s1Path.reverse()
	s2Path.reverse()
	startIS1 = fromPointer[4][0]-1
	startIS2 = fromPointer[4][1]-1
	if(startIS1>startIS2):
		s1Start = s1[0:startIS1]
		s2Start = ['-']*len(s1Start)
	else:
		s2Start = s2[0:startIS2]
		s1Start = ['-']*len(s2Start)

	endIS1 = maxEnd[4][0]
	endIS2 = maxEnd[4][1]
	if(endIS1<endIS2):
		s1End = s1[endIS1:]
		s2End = ['-']*len(s1End)
	else:
		s2End = s2[endIS2:]
		s1End = ['-']*len(s2End)
	print(''.join(s1Start+s1Path+s1End))
	print(''.join(s2Start+s2Path+s2End))

When the function is run, to run it on the first 35 bases in B and last 35 bases in C (from 2A)  i used the string split function on the stored values from above. Note: the gap penalty is given as positive, since it is subtracted in the code.

In [98]:

overlapAlignment(list(seqA.stringRepr())[:36],list(seqB.stringRepr()[-35:]), [[2,-2,-2,-2],[-2,2,-2,-2],[-2,-2,2,-2],[-2,-2,-2,2]], 3)

ATTTTCACCTGCCCCATCGAGGTCGACCCGCCGACG-----------
---------TG-CATAT-GA-GAGGCCACGCGGACGAGATCACCTTC


In [99]:
print("gap score of -4")
overlapAlignment(list(seqA.stringRepr())[:36],list(seqB.stringRepr()[-35:]), [[2,-2,-2,-2],[-2,2,-2,-2],[-2,-2,2,-2],[-2,-2,-2,2]], 4)
print("gap score of -2")
overlapAlignment(list(seqA.stringRepr())[:36],list(seqB.stringRepr()[-35:]), [[2,-2,-2,-2],[-2,2,-2,-2],[-2,-2,2,-2],[-2,-2,-2,2]], 2)
print("gap score of -1")
overlapAlignment(list(seqA.stringRepr())[:36],list(seqB.stringRepr()[-35:]), [[2,-2,-2,-2],[-2,2,-2,-2],[-2,-2,2,-2],[-2,-2,-2,2]], 1)

gap score of -4
ATTTTCACCTGCCCCATCGAGGTCGACCCGCCGACG-----------
---------TG-CATAT-GA-GAGGCCACGCGGACGAGATCACCTTC
gap score of -2
ATTTTCACCTGCCCCATCGAG-GTCGACC-CGCCGACG-----------
---------TG-CATAT-GAGAG--G-CCACGCGGACGAGATCACCTTC
gap score of -1
ATTTTCACCTGCCCCAT-CGAG-GTCGACC-CGCCGACG-----------
---------TG---CATATGAGAG--G-CCACGCGGACGAGATCACCTTC


The harsher the gap penalty is compared to the penalty for mismatching bases, the algorithm will try to replace more gaps with mismatched bases. The best result will depend on what the alignment is going to be used for, if for example you wanted to change one sequence into the other, and gaps are easy to do and changing one base into anouther is hard, you would want a low gap penalty and a high mismatch penalty, so something like the gap sequence of -1 would most likly be the best.

For string matching, just visually comparing the alignments, the lower gap penalty compared to mismatch penalty also seems the best. Gaps are easy to visually skip over, but mismatched letters make the alignment seem worse even if the alogrithmic scores are better. So in my opinion a value for the gap penalty of -1 seems the best. 